In [20]:
! pip install pynput

  Using cached pynput-1.8.1-py2.py3-none-any.whl.metadata (32 kB)
Using cached pynput-1.8.1-py2.py3-none-any.whl (91 kB)


In [1]:
import numpy as np
import cv2
import threading
import queue
import time
from pynput import keyboard



тест отдельно по камерам

In [2]:
idx_cam = 0
cap = cv2.VideoCapture(idx_cam)
if not cap.isOpened():
    print(f"Error open camera {idx_cam}")
    # return

fps = int(cap.get(cv2.CAP_PROP_FPS))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
font = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX

# fourcc = cv2.VideoWriter_fourcc(*'XVID')  #MP4V   пропуск кадров
# output_video = rf'{path_save_video}\video_cam_{idx_cam}.avi'  # пропуск кадров
# writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))  # пропуск кадров

while True:
    ret, frame = cap.read()
    if not ret:
        print(f'Lost connetct with camera id: {idx_cam}')
        break
    text = f'{str(fps)} : {str(height)} : {str(width)} '
    cv2.putText(frame, text, (10, 30), font, 1.0, (255, 0, 0), 1)

    # writer.write(frame)  # пропуск кадров

    # clearing the queue (чтоб не тормозила камера) so that the camera doesn't slow down
    

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Was pressed key quit")
        break

    cv2.imshow(f'camera_{str(idx_cam)}', frame)

cap.release()
cv2.destroyAllWindows()

Was pressed key quit


define function for visualize from camera

In [2]:
idx_cam_1 = 0
idx_cam_2 = 1

frame_queue1 = queue.Queue(maxsize=1)
frame_queue2 = queue.Queue(maxsize=1)

writers={}

def capture_camera(idx_cam: int, frame_queue):
    cap = cv2.VideoCapture(idx_cam)
    if not cap.isOpened():
        print(f"Error open camera {idx_cam}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    font = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX

    # fourcc = cv2.VideoWriter_fourcc(*'XVID')  #MP4V   пропуск кадров
    # output_video = rf'{path_save_video}\video_cam_{idx_cam}.avi'  # пропуск кадров
    # writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))  # пропуск кадров
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print(f'Lost connetct with camera id: {idx_cam}')
            break
        text = f'{str(fps)} : {str(height)} : {str(width)} '
        cv2.putText(frame, text, (10, 30), font, 1.0, (255, 0, 0), 1)

        # writer.write(frame)  # пропуск кадров

        # clearing the queue (чтоб не тормозила камера) so that the camera doesn't slow down
        if not frame_queue.empty():
            frame_queue.get()
        
        frame_queue.put((idx_cam, frame))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Was pressed key quit")
            break

        # cv2.imshow(f'camera_{str(idx_cam)}', frame)
    
    cap.release()
    # writer.release()   # пропуск кадров
    # cv2.destroyAllWindows()


### Example with threading

In [3]:
# start threading
idx_cam_1 = 0
idx_cam_2 = 1

frame_queue1 = queue.Queue(maxsize=1)
frame_queue2 = queue.Queue(maxsize=1)

writers={}

def capture_camera(idx_cam: int, frame_queue):
    cap = cv2.VideoCapture(idx_cam)
    if not cap.isOpened():
        print(f"Error open camera {idx_cam}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    font = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX

    # fourcc = cv2.VideoWriter_fourcc(*'XVID')  #MP4V   пропуск кадров
    # output_video = rf'{path_save_video}\video_cam_{idx_cam}.avi'  # пропуск кадров
    # writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))  # пропуск кадров
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print(f'Lost connetct with camera id: {idx_cam}')
            break
        text = f'{str(fps)} : {str(height)} : {str(width)} '
        cv2.putText(frame, text, (10, 30), font, 1.0, (255, 0, 0), 1)

        # writer.write(frame)  # пропуск кадров

        # clearing the queue (чтоб не тормозила камера) so that the camera doesn't slow down
        if not frame_queue.empty():
            frame_queue.get()
        
        frame_queue.put((idx_cam, frame))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Was pressed key quit")
            break

        # cv2.imshow(f'camera_{str(idx_cam)}', frame)
    
    cap.release()
    # writer.release()   # пропуск кадров
    # cv2.destroyAllWindows()



def main():

    path_save1 = r'record_cam1'
    path_save2 = r'record_cam2'

    t1 = threading.Thread(target=capture_camera, args=(idx_cam_1, frame_queue1), daemon=True)
    t2 = threading.Thread(target=capture_camera, args=(idx_cam_2, frame_queue2), daemon=True)

    t1.start()
    t2.start()

    # Main loop visualize
    try:
        while True:
            current_time = time.strftime("%Y%m%d_%H%M%S")
            

            if not frame_queue1.empty():
                idx, frame = frame_queue1.get()
                cv2.imshow(f'Camera_{idx}', frame)

                #Initialize VideoWriter 
                if idx not in writers:
                    filename = fr'{path_save1}\camera_{idx}_{current_time}.mp4'
                    fourcc = cv2.VideoWriter_fourcc(*'H264')  # XVID
                    writers[idx] = cv2.VideoWriter(filename, fourcc, 30, (frame.shape[1], frame.shape[0]))
                    print(f'Record start: {filename}')
                
                writers[idx].write(frame)
            
            if not frame_queue2.empty():
                idx, frame = frame_queue2.get()
                cv2.imshow(f'Camera_{idx}', frame)

                #init vwr
                if idx not in writers:
                    filename = fr'{path_save2}\camera_{idx}_{current_time}.avi'
                    fourcc = cv2.VideoWriter_fourcc(*'XVID')
                    writers[idx] = cv2.VideoWriter(filename, fourcc, 30, (frame.shape[1], frame.shape[0]))
                    print(f'Record avi start: {filename}')
                
                writers[idx].write(frame)

            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            time.sleep(0.01) 
    except KeyboardInterrupt:
        pass
    for writer in writers.values():
        writer.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


Record start: record_cam1\camera_0_20250724_143334.mp4
Record avi start: record_cam2\camera_1_20250724_143334.avi


Если камеры подключены по сети (IP-камеры), то cv2.VideoCapture(idx_cam) может работать нестабильно — лучше использовать cv2.CAP_FFMPEG или cv2.CAP_DSHOW в зависимости от платформы.

### use multiprocessing

In [4]:
import cv2
import multiprocessing as mp
import time



In [2]:
def capture_camera_process(idx_cam: int, queue: mp.Queue):
    """function to work in diferent process - capture frame and sent in queue"""
    cap = cv2.VideoCapture(idx_cam)
    if not cap.isOpened():
        print(f'Cam {idx_cam} not opened')
        return
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"[Процесс {idx_cam}] Подключено: {width}x{height} @ {fps}fps")


    while True:
        ret, frame = cap.read()
        if not ret:
            print(f'Not get frame')
            break
        # Добавляем текст на кадре (можно и в главном процессе, но так легче)
        text = f'Cam {idx_cam} | {width}x{height} @ {fps}fps'
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # if queue is full
        if not queue.empty():
            queue.get() # cleareing queue
        queue.put((idx_cam, frame))

        
        time.sleep(0.01)
    cap.release()
    queue.put((idx_cam, None)) # Signal to end
    print(f'Process cam {idx_cam} END.')

In [3]:
mp.set_start_method('spawn', force=True)  # Критично для Windows/macOS
idx_cam_1 = 0
idx_cam_2 = 1

# Создаём очереди для передачи кадров из процессов
queue1 = mp.Queue(maxsize=1)
queue2 = mp.Queue(maxsize=1)

# Запускаем процессы
proc1 = mp.Process(target=capture_camera_process, args=(idx_cam_1, queue1), daemon=True)
proc2 = mp.Process(target=capture_camera_process, args=(idx_cam_2, queue2), daemon=True)

proc1.start()
proc2.start()

print('Start processes to capture cadrs. Press "q" to exit')
try:
    while True:
        frame1 = None
        frame2 = None

        if not queue1.empty():
            cam_id, frame1 = queue1.get()
            
            if frame1 is None:
                print('Camera 1 is stop')
                break
        
        if not queue2.empty():
            cam_id, frame2 = queue2.get()
            if frame2 is None:
                print('Camera 2 is stop')
                break

        if frame1 is not None:
            cv2.imshow(f'Camera_{cam_id}', frame1)
        if frame2 is not None:
            cv2.imshow(f'Camera {cam_id}', frame2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print('Exit by command user')
            break
        time.sleep(0.01)
except KeyboardInterrupt:
    print('Stopped USER')

finally:
    # stoped processes
    cv2.destroyAllWindows()
    proc1.terminate()
    proc2.terminate()
    proc1.join(timeout=1)
    proc2.join(timeout=1)
    cv2.destroyAllWindows()
    print('Resourses to free')


Start processes to capture cadrs. Press "q" to exit
Stopped USER
Resourses to free


In [4]:
print('Hello')

Hello


Пример изменения источника камеры по нажатию клавиши, НЕ ТАК КАК НАДО работает, просто пример

In [ ]:

cam = "camera_1"

def on_press(key):
    global cam
    try:
        if key.char == ' ': #
            if cam == 'camera_1':
                cam == 'camera_2'
                get_video(idx_cam_2)
            else:
                get_video(idx_cam_1)
    except Exception as e:
        print(f'Error: {e}')

listener = keyboard.Listener(on_press=on_press)
listener.start()

print(f'Press space for change camera. Ctrl+C for exit.')

try:
    while True:
        if cam == 'camera_1':
            print(f'Release camera 1')
            get_video(idx_cam_1)
        else:
            print(f'Release camera 2')
            get_video(idx_cam_2)
except KeyboardInterrupt:
    listener.stop()
    print("\nExit")

Press space for change camera. Ctrl+C for exit.
Release camera 1
Press quit


Если камеры подключены по сети (IP-камеры), то cv2.VideoCapture(idx_cam) может работать нестабильно — лучше использовать cv2.CAP_FFMPEG или cv2.CAP_DSHOW в зависимости от платформы.